In [ ]:
import numpy as np
import pandas as pd
train_df = pd.read_csv('/kaggle/input/train.csv',index_col=0)
submission_df = pd.read_csv('/kaggle/input/sample_submission.csv',index_col=0)
testFeaturesData = pd.read_csv('/kaggle/input/test.csv',index_col=0)
# train_df = train_df.dropna()

In [ ]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

train_df['distance'] = haversine_array(train_df['pick_lat'],train_df['pick_lon'], train_df['pick_lat'], train_df['drop_lon']) + haversine_array(train_df['pick_lat'], train_df['pick_lon'], train_df['drop_lat'], train_df['pick_lon'])
testFeaturesData['distance'] = haversine_array(testFeaturesData['pick_lat'],testFeaturesData['pick_lon'], testFeaturesData['pick_lat'], testFeaturesData['drop_lon']) + haversine_array(testFeaturesData['pick_lat'], testFeaturesData['pick_lon'], testFeaturesData['drop_lat'], testFeaturesData['pick_lon'])
train_df = train_df.drop(columns=['pick_lat','pick_lon','drop_lat','drop_lon'])
testFeaturesData = testFeaturesData.drop(columns=['pick_lat','pick_lon','drop_lat','drop_lon'])

In [ ]:
a = train_df['pickup_time'].str.slice(0, 15)
a= pd.to_datetime(a, utc=True, format='%m/%d/%Y %H:%M')
a.dt.hour
train_df['phour'] = a.dt.hour
train_df['pday'] = a.dt.day
train_df['pmonth'] = a.dt.month
train_df['pyear'] = a.dt.year

b = testFeaturesData['pickup_time'].str.slice(0, 15)
b= pd.to_datetime(b, utc=True, format='%m/%d/%Y %H:%M')
b.dt.hour
testFeaturesData['phour'] = b.dt.hour
testFeaturesData['pday'] = b.dt.day
testFeaturesData['pmonth'] = b.dt.month
testFeaturesData['pyear'] = b.dt.year

c = train_df['drop_time'].str.slice(0, 15)
c= pd.to_datetime(c, utc=True, format='%m/%d/%Y %H:%M')
c.dt.hour
train_df['dhour'] = c.dt.hour
train_df['dday'] = c.dt.day
train_df['dmonth'] = c.dt.month
train_df['dyear'] = c.dt.year

d = testFeaturesData['drop_time'].str.slice(0, 15)
d= pd.to_datetime(d, utc=True, format='%m/%d/%Y %H:%M')
d.dt.hour
testFeaturesData['dhour'] = d.dt.hour
testFeaturesData['dday'] = d.dt.day
testFeaturesData['dmonth'] = d.dt.month
testFeaturesData['dyear'] = d.dt.year
# train_df['timeDiff']=(pd.to_datetime(train_df['drop_time'], utc=True, format='%m/%d/%Y %H:%M')-pd.to_datetime(train_df['pickup_time'], utc=True, format='%m/%d/%Y %H:%M')).dt.seconds / 60.0
# testFeaturesData['timeDiff']=(pd.to_datetime(testFeaturesData['drop_time'], utc=True, format='%m/%d/%Y %H:%M')-pd.to_datetime(testFeaturesData['pickup_time'], utc=True, format='%m/%d/%Y %H:%M')).dt.seconds /60.0
train_df = train_df.drop('pickup_time', axis=1)
testFeaturesData = testFeaturesData.drop('pickup_time', axis=1)
train_df = train_df.drop('drop_time', axis=1)
testFeaturesData = testFeaturesData.drop('drop_time', axis=1)


In [ ]:
trainLabelsData = train_df.filter(['label'], axis=1)
trainFeaturesData =  train_df.drop('label', axis=1)

In [ ]:
label_classes = {'correct': 1,
                'incorrect': 0
                          }
trainLabelsData['label'] = trainLabelsData['label'].map(label_classes)

In [ ]:
numerical_cols = np.array(['duration', 'meter_waiting','meter_waiting_fare', 'meter_waiting_till_pickup','fare','distance','dhour','dday','dmonth','dyear'])
boolean_cols = np.array(['additional_fare'])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

In [ ]:
boolean_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='mean'))
])
# constant

numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='most_frequent'))
])
# most_frequent
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numerical_cols),
        ("boolean", boolean_steps, boolean_cols)
    ],
    remainder = "drop"
)

In [ ]:
estimator = XGBClassifier(max_depth=4, n_estimators=240, subsample=0.9)

In [ ]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimator),
])

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(trainFeaturesData, trainLabelsData, test_size=0.3, random_state=1) 

In [ ]:
# full_pipeline.fit(X_train, y_train.values.ravel())
# preds_ = full_pipeline.predict(X_eval)
# preds = [round(value) for value in preds_]

In [ ]:
estimator.fit(X_train, y_train.values.ravel())
preds_ = estimator.predict(X_eval)
preds = [round(value) for value in preds_]

In [ ]:
y_preds = pd.DataFrame(
    {
        "label": preds
    },
    index = y_eval.index
)

In [ ]:
f1_score(y_eval, y_preds, average='macro')

In [ ]:
full_pipeline.fit(trainFeaturesData, trainLabelsData.values.ravel())
results_ = full_pipeline.predict(testFeaturesData)
results =[round(value) for value in results_]

In [ ]:
np.testing.assert_array_equal(testFeaturesData.index.values, 
                              submission_df.index.values)
submission_df["prediction"] = results
submission_df.to_csv('my_submission.csv', index=True)